In [1]:
from fastai.text import *

In [2]:
PATH = Path('data')
POEMS = PATH/'manyoshu_2.txt'
WIKI = PATH/'ja.text8.txt'

In [69]:
poems = POEMS.open().read().strip().split(',')
poems = [poem.lstrip(' ').replace('\n','').replace('"','').replace("'","") for poem in poems]
wiki = WIKI.open().read().replace('\n','').replace('"','').replace("'","").split()

In [70]:
len(poems), poems[:3], len(wiki)

(4430,
 [' 篭 も よみ 篭 持ち 堀 串 も よみ 堀 串 持ち この 岡 に 菜摘 ます子 家 聞か な 告 ら さ ね そらみつ 大和 の 国 は おしなべて 我 れ こそ 居れ しき なべて 我 れ こそ 座せ 我 れこ そば 告 ら め 家 を も 名 を も ',
  ' 大和 に は 群 山 あれ ど とり よろ ふ 天 の 香具 山登り 立ち 国見 を すれ ば 国原 は 煙 立ち 立つ 海原 は 鴎 立ち 立つ うまし 国 ぞ 蜻蛉 島 大和 の 国 は ',
  ' やすみ し し 我が 大君 の 朝 に は 取り 撫で た まひ 夕 に は い 寄り 立たし しみ 執ら し の 梓 の 弓 の 中 弭 の 音 す なり 朝 猟 に 今 立たす らし 夕 猟 に 今 立た すら しみ 執ら し の 梓 の 弓 の 中 弭 の 音 す なり '],
 16882329)

In [68]:
tok_poems = [poem.split() for poem in poems]
tok_wiki = wiki[:120000]

In [6]:
tok_poems[:2]

[["'",
  '篭',
  'も',
  'よみ',
  '篭',
  '持ち',
  '堀',
  '串',
  'も',
  'よみ',
  '堀',
  '串',
  '持ち',
  'この',
  '岡',
  'に',
  '菜摘',
  'ます子',
  '家',
  '聞か',
  'な',
  '告',
  'ら',
  'さ',
  'ね',
  'そらみつ',
  '大和',
  'の',
  '国',
  'は',
  'おしなべて',
  '我',
  'れ',
  'こそ',
  '居れ',
  'しき',
  'なべて',
  '我',
  'れ',
  'こそ',
  '座せ',
  '我',
  'れこ',
  'そば',
  '告',
  'ら',
  'め',
  '家',
  'を',
  'も',
  '名',
  'を',
  'も',
  "'"],
 ["'",
  '大和',
  'に',
  'は',
  '群',
  '山',
  'あれ',
  'ど',
  'とり',
  'よろ',
  'ふ',
  '天',
  'の',
  '香具',
  '山登り',
  '立ち',
  '国見',
  'を',
  'すれ',
  'ば',
  '国原',
  'は',
  '煙',
  '立ち',
  '立つ',
  '海原',
  'は',
  '鴎',
  '立ち',
  '立つ',
  'うまし',
  '国',
  'ぞ',
  '蜻蛉',
  '島',
  '大和',
  'の',
  '国',
  'は',
  "'"]]

In [3]:
trn_poems,val_poems = sklearn.model_selection.train_test_split(tok_poems, test_size=0.2)
trn_wiki,val_wiki = sklearn.model_selection.train_test_split(tok_wiki, test_size=0.2)

pickle.dump(trn_poems, open(PATH/'trn_poems_ja.pkl', 'wb'))
pickle.dump(val_poems, open(PATH/'val_poems_ja.pkl', 'wb'))

pickle.dump(trn_wiki, open(PATH/'trn_wiki_ja.pkl', 'wb'))
pickle.dump(val_wiki, open(PATH/'val_wiki_ja.pkl', 'wb'))

trn_poems,val_poems = pickle.load(open(PATH/'trn_poems_ja.pkl', 'rb')), pickle.load(open(PATH/'val_poems_ja.pkl', 'rb')) 
trn_wiki,val_wiki = pickle.load(open(PATH/'trn_wiki_ja.pkl', 'rb')), pickle.load(open(PATH/'val_wiki_ja.pkl', 'rb'))

In [6]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [7]:

trn_poems = list(chunks([tok for poem in trn_poems for tok in poem if tok not in [' ','"',"'"]],10))
val_poems = list(chunks([tok for poem in val_poems for tok in poem if tok not in [' ','"',"'"]],10))

trn_wiki = list(chunks(trn_wiki,10))
val_wiki = list(chunks(val_wiki,10))

In [8]:
import collections

def toks2ids(tok, pre):
    freq = collections.Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(30000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    pickle.dump(itos, open(PATH/f'{pre}_itos.pkl', 'wb'))
    return itos,stoi

In [9]:
itos,stoi = toks2ids(trn_poems, 'ja')
itos_d,stoi_d = toks2ids(trn_wiki+trn_poems, 'ja')

In [7]:
itos = pickle.load(open(PATH/'ja_itos.pkl', 'rb'))
stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})

In [10]:
len(stoi), len(stoi_d)

(5167, 17482)

In [11]:
trn_p_ids = np.array([[0] + [stoi[o] if o in stoi else 3 for o in p] + [2] for p in trn_poems])
val_p_ids = np.array([[0] + [stoi[o] if o in stoi else 3 for o in p] + [2] for p in val_poems])
trn_w_ids = np.array([[0] + [stoi_d[o] if o in stoi_d else 3 for o in p] + [2] for p in trn_wiki])
val_w_ids = np.array([[0] + [stoi_d[o] if o in stoi_d else 3 for o in p] + [2] for p in val_wiki])

In [12]:
len(trn_p_ids), len(val_p_ids), len(trn_w_ids), len(val_w_ids)

(8122, 2011, 9600, 2400)

In [17]:
sum(len(i) for i in trn_p_ids)/len(trn_p_ids)

11.999546998867498

In [13]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    header = fin.readline().split()
    n, d = header[0], header[1]
    data = {}
    #partitions = [fin[:]]
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:], dtype=float)
    return data, n, d

In [14]:
ja_vecs, vs, dim_ja_vec = load_vectors(str(PATH/'cc.ja.300.vec'))

In [15]:
class GANDataset(Dataset):
    def __init__(self, x): 
        self.x = x
    def __getitem__(self, idx):
        seq_len = len(self.x[idx])
        return A(self.x[idx][:seq_len//2]), A(self.x[idx][seq_len//2:])
    def __len__(self): return len(self.x)
    
class DiscDataset(Dataset):
    def __init__(self, real, fake): 
        self.real = real
        self.fake = fake
    def __getitem__(self, idx):
        return A(self.real[idx]), A(self.fake[idx])
    def __len__(self): return len(self.real)

In [16]:
trn_seq2seq_ds = GANDataset(trn_p_ids)
val_seq2seq_ds = GANDataset(val_p_ids)

trn_gan_ds = GANDataset(trn_p_ids)
val_gan_ds = GANDataset(val_p_ids)

trn_disc_ds = DiscDataset(trn_p_ids, trn_w_ids[:len(trn_p_ids)])
val_disc_ds = DiscDataset(val_p_ids, val_w_ids[:len(trn_p_ids)])

In [17]:
bs=32

trn_samp = SortishSampler(trn_p_ids, lambda x: len(trn_p_ids[x]), bs)
val_samp = SortSampler(val_p_ids, lambda x: len(val_p_ids[x]))

trn_seq2seq_dl = DataLoader(trn_seq2seq_ds,batch_size=bs,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True, sampler=trn_samp)
val_seq2seq_dl = DataLoader(val_seq2seq_ds,batch_size=bs,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True, sampler=val_samp)

trn_gan_dl = DataLoader(trn_gan_ds,batch_size=bs,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True, sampler=trn_samp)
val_gan_dl = DataLoader(val_gan_ds,batch_size=bs,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True, sampler=val_samp)

trn_disc_dl = DataLoader(trn_disc_ds,batch_size=bs//2,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True)
val_disc_dl = DataLoader(val_disc_ds,batch_size=bs//2,pad_idx=1,num_workers=1,pre_pad=False, transpose_y=True, transpose=True)

In [18]:
x, y = next(iter(trn_gan_dl))
x.size(), y.size()

(torch.Size([6, 32]), torch.Size([6, 32]))

In [19]:
def rand_t(*sz): return torch.randn(sz)/math.sqrt(sz[0])
def rand_p(*sz): return nn.Parameter(rand_t(*sz))

In [20]:
def repackage_hidden(h):
    return h.detach() if isinstance(h, torch.Tensor) else tuple(repackage_hidden(v) for v in h)

In [21]:
class Seq2SeqAttention(nn.Module):
    def __init__(self, vecs, itos, em_sz, nh, nl=2, dropf=1.0):
        super().__init__()
        self.emb_enc = create_emb(vecs, itos, em_sz)
        self.nl,self.nh = nl,nh
        self.lstm_enc = nn.LSTM(em_sz, nh, num_layers=nl, dropout=0.25*dropf, bidirectional=True)
        self.out_enc = nn.Linear(nh*2, em_sz, bias=False)
        self.drop_enc = nn.Dropout(0.25*dropf)
        self.emb_dec = create_emb(vecs, itos, em_sz)
        self.lstm_dec = nn.LSTM(em_sz, em_sz, num_layers=nl, dropout=0.1)
        self.emb_enc_drop = nn.Dropout(0.15*dropf)
        self.out_drop = nn.Dropout(0.35*dropf)
        self.out = nn.Linear(em_sz, len(itos))
        self.out.weight.data = self.emb_dec.weight.data

        self.W1 = rand_p(nh*2, em_sz)
        self.l2 = nn.Linear(em_sz, em_sz)
        self.l3 = nn.Linear(em_sz+nh*2, em_sz)
        self.V = rand_p(em_sz)

    def forward(self, inp, y=None,sampling=False):
        sl,bs = inp.size()
        h_n, c_n = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, (h_n, c_n) = self.lstm_enc(emb, (h_n, c_n))
        h_n = h_n.view(2,2,bs,-1).permute(0,2,1,3).contiguous().view(self.nl,bs,-1)
        h_n = self.out_enc(self.drop_enc(h_n))
        c_n = c_n.view(2,2,bs,-1).permute(0,2,1,3).contiguous().view(self.nl,bs,-1)
        c_n = self.out_enc(self.drop_enc(c_n))
        pg_loss=0
        dec_inp = V(torch.zeros(bs).long())
        res,attns = [],[]
        samples = V(torch.zeros(sl, bs).long())
        w1e = enc_out @ self.W1
        for i in range(sl):
            w2h = self.l2(h_n[-1])
            u = torch.tanh(w1e + w2h)
            a = F.softmax(u @ self.V, 0)
            attns.append(a)
            Xa = (a.unsqueeze(2) * enc_out).sum(0)
            emb = self.emb_dec(dec_inp)
            wgt_enc = self.l3(torch.cat([emb, Xa], 1))
            outp, (h_n, c_n) = self.lstm_dec(wgt_enc.unsqueeze(0), (h_n,c_n))
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            if not sampling:
                dec_inp = V(outp.data.max(1)[1])
            else:
                outp = F.log_softmax(outp, dim=1)
                outp = torch.multinomial(torch.exp(outp), 1)
                samples[i, :] = outp.view(-1).data
                dec_inp = V(outp.view(-1))
            if (dec_inp==1).all(): break
            if (y is not None):
                if i>=len(y): break
                dec_inp = y[i]
        h_n = repackage_hidden(h_n)
        c_n = repackage_hidden(c_n)
        
        return torch.stack(res), samples
    
    
    def initHidden(self, bs): return V(torch.zeros(self.nl*2, bs, self.nh)), V(torch.zeros(self.nl*2, bs, self.nh))

In [22]:
def seq2seq_loss(input, target, kld_weight=0):
    decoded = input
    sl, bs = target.size()
    sl_in,bs_in,nc = decoded.size()
    if sl>sl_in: decoded = F.pad(decoded, (0,0,0,0,0,sl-sl_in))
    decoded = decoded[:sl]
    return F.cross_entropy(decoded.view(-1,nc), target.view(-1))

In [24]:
class TextDicriminator(nn.Module):
    def __init__(self,vecs,itos,em_sz,nh,nl=2, dropf=1):
        super().__init__()
        #encoder
        self.emb_enc = create_emb(vecs,itos,em_sz)
        self.emb_enc_drop = nn.Dropout(0.15*dropf)
        self.nl,self.nh, self.vs = nl,nh,len(itos)
        self.lstm_enc = nn.LSTM(em_sz, nh, num_layers=nl, dropout=0.25*dropf, bidirectional=True)
        self.out_drop = nn.Dropout(0.35*dropf)
        #attention
        self.W1 = rand_p(nh*2, em_sz)
        self.l2 = nn.Linear(nh*2, em_sz)
        #self.l3 = nn.Linear(nh*4, em_sz)
        self.V = rand_p(em_sz)
        
        #classifier
        #self.out_enc = nn.Linear(nh*3*2, em_sz)
        #self.nonlin = nn.Tanh()
        self.out = nn.Linear(nh*2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inp, y=None):
        #encode
        sl,bs = inp.size()
        hidden = self.initHidden(bs) # nl*bs*nh
        emb = self.emb_enc_drop(self.emb_enc(inp)) #sl*bs*em_sz
        enc_out, (h_n, c_n) = self.lstm_enc(emb,hidden) #enc_out: sl*bs*nh, hidden: nl*bs*nh
        #classify
        h_n = h_n.view(2,2,bs,-1).permute(0,2,1,3).contiguous().view(self.nl,bs,-1)
        #hidden = torch.cat([enc_out.max(0)[0],enc_out.sum(0),h_n[-1]],1)
        #c = self.nonlin(self.out_enc(hidden))
        #c = self.sigmoid(self.out(self.out_drop(c)))
        out = []
        w1e = enc_out @ self.W1
        #for i in range(sl):
        w2h = self.l2(h_n[-1])
        u = torch.tanh(w1e + w2h)
        a = F.softmax(u @ self.V, 0)
        #out.append(a)
        Xa = (a.unsqueeze(2) * enc_out).sum(0)
            #emb = self.emb_enc(inp[i])
        #wgt_enc = self.l3(torch.cat([h_n[-1], Xa], 1))
            #c = self.nonlin(self.out_enc(hidden))
        c = self.sigmoid(self.out(self.out_drop(Xa)))
            #out.append(self.sigmoid(wgt_enc))
            #enc = self.nonlin(self.out_enc(torch.cat([enc_out.max(0)[0],enc_out.sum(0),enc_out[i]],1)))
            #out.append(self.sigmoid(self.out(self.out_drop(enc))))
        #h_n = repackage_hidden(h_n)
        #c_n = repackage_hidden(c_n)
        #return out
        return  c, a

    def initHidden(self, bs): return V(torch.zeros(self.nl*2, bs, self.nh)), V(torch.zeros(self.nl*2, bs, self.nh))

In [25]:
def create_emb(vecs, itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    wgts = emb.weight.data
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = torch.from_numpy(vecs[w])
        except: miss.append(w)
    print(len(miss),miss[5:10])
    return emb

In [26]:
disc = TextDicriminator(ja_vecs, itos_d, int(dim_ja_vec), 128)
gen = Seq2SeqAttention(ja_vecs, itos, int(dim_ja_vec), 128)
gen.cuda(); 
disc.cuda();

924 ['居ら', '羨し', '。『', '按察', 'むよしもがも']
347 ['居ら', '羨し', 'むよしもがも', '渡ら', 'ほえ']
347 ['居ら', '羨し', 'むよしもがも', '渡ら', 'ほえ']


In [27]:
from torch import optim

optimizerD = optim.RMSprop(disc.parameters(), lr = 3e-3)
optimizerG = optim.Adam(gen.parameters(), lr = 3e-3, betas=(0.7, 0.8))

In [28]:
def train_gen(gen, epochs, trn_dl, val_dl, crit, opt):
    n = epochs
    for epoch in range(epochs):
        with tqdm(total=len(trn_dl)) as pbar:
            gen.train()
            for i, ds in enumerate(trn_dl):
                x, y = ds
                sl, bs = x.size()
                fake = gen(x,y)
                gen.zero_grad()
                gen_loss = creative_loss = crit(fake[0],y)
                gen_loss.backward()
                opt.step()
                torch.nn.utils.clip_grad_norm_(gen.parameters(), 8.)
                pbar.update()
        print(f'Epoch {epoch}:')
        print(f'Loss_G {creative_loss.data.item()} Ppx {torch.exp(creative_loss)}')
        gen.eval()
        for i, ds in enumerate(val_dl):
            x, y = ds
            sl, bs = x.size()
            fake = gen(x)
            gen_loss = creative_loss = crit(fake[0],y)
        print(f'Val Loss_G {creative_loss.data.item()} Ppx {torch.exp(creative_loss)}')

In [147]:
train_gen(gen,10, trn_gan_dl, val_gan_dl, seq2seq_loss, optimizerG)

100%|██████████| 2538/2538 [02:41<00:00, 15.72it/s]
Epoch 0:
Loss_G 4.226513862609863 Ppx 68.47808837890625
Val Loss_G 5.777705669403076 Ppx 323.0172119140625
100%|██████████| 2538/2538 [02:41<00:00, 16.18it/s]
Epoch 1:
Loss_G 4.537458896636963 Ppx 93.4530258178711
Val Loss_G 6.042873859405518 Ppx 421.10150146484375
100%|██████████| 2538/2538 [02:41<00:00, 15.70it/s]
Epoch 2:
Loss_G 5.070113658905029 Ppx 159.19241333007812
Val Loss_G 6.283552169799805 Ppx 535.6881103515625
 86%|████████▌ | 2183/2538 [01:58<00:14, 23.87it/s]


KeyboardInterrupt: 

In [148]:
torch.save(gen.state_dict(), PATH/'models/seq2seq_ja2.h5')

In [29]:
gen.load_state_dict(torch.load(PATH/'models/seq2seq_ja2.h5', map_location=lambda storage, loc: storage))

In [30]:
def produce_out(val_dl, model, use_cuda=True):
    model.eval()
    out = []
    loss = 0
    #val_dl.sampler=None
    for i, ds in enumerate(val_dl):
        x, y = ds
        bptt, bs = x.size()
        y = y.view(bptt, bs)
        #x = x.long()
        if use_cuda:
            x = x.cuda()
        probs = model(x)
        loss = seq2seq_loss(probs[0], y)
        preds = np.array(probs[0].topk(2)[1])
        for i in range(x.size(1)):
            seed = ' '.join([itos[o] for o in x[:,i] if o not in [0,1,2,3]])
            actual = ' '.join([itos[o] for o in y[:,i] if o not in [0,1,2,3]])
            fake = ' '.join([itos[o[0]] if o[0] not in [0,1,2,3] else itos[o[-1]] for o in preds[:,i]])
            out.append([seed,actual,fake])
    print(torch.exp(loss))
    return out

In [31]:
out_seq2seq = produce_out(val_gan_dl, gen)
pickle.dump(out_seq2seq, (PATH/'seq2seq_ja_small.pkl').open('wb'))

tensor(772.9237, device='cuda:0', grad_fn=<ExpBackward>)


In [32]:
out_seq2seq = pickle.load((PATH/'seq2seq_ja_small.pkl').open('rb'))
out_seq2seq

[['霍公 鳥 聞け ども 飽か', 'ず 網 捕り に 捕り', 'ぬ か も 君 が に'],
 ['て な 離れ ず', '鳴く が ね やすみ し', 'も 君 が 家 の に'],
 ['し 我が 大君 の 敷き', 'ませる 国 の 中 に', 'て も 君 が 恋 ひ'],
 ['は 都 し 思 ほ', 'ゆ 山 高み 谷 辺', 'ゆる かも 我が 恋 ひ に'],
 ['に 延 へる 玉葛 絶', 'ゆる 時 なく 見 むよしもがも', 'ゆる こと なく も が ば'],
 ['我が 宿 に 植 ゑ', '生 ほし たる 秋 萩', 'に し あれ ば 君 に'],
 ['を 誰 れ か 標', '刺す 我 れ に 知ら', 'は 恋 ひ つつ あら ず'],
 ['え ず お ほなら ばか', 'も かもせ む を 畏', 'も 我 れ は 恋 ひ'],
 ['み と 振り たき 袖', 'を 忍び て ある かも', 'に し あれ ば 君 に'],
 ['霰 降り 遠 つ 淡海', 'の 吾 跡 川 楊', 'の 山 に も 我が に'],
 ['刈れ ども また も 生', 'ふと いふ 吾 跡 川', 'ひ に けり 我が 恋 ひ'],
 ['楊 世の中 は 恋 繁し', '梅 の 花 に', 'な ば 君 が 家 に'],
 ['も なら まし もの を', '守 大伴 大夫 世間', '我が 恋 ひ つつ あら ず'],
 ['の すべ なき もの は', '年月 は 流る る ごとし', '恋 ひ つつ あら む に'],
 ['とり 続き 追 ひ 来る', 'もの は 百 種 に', 'と も 君 が 恋 ひ'],
 ['迫 め 寄り 来る 娘', '子 ら が 娘 子', 'の 音 のみ し を も'],
 ['さ 唐 玉 を', '手本 に 巻か し 白妙', 'に は 恋 ひ つつ も'],
 ['の 袖 振り 交 は', 'し 紅 の 赤 裳裾', 'し 妹 が 家 に に'],
 ['引き よ ち 子 ら', 'と 手 携 は り', 'が 恋 ひ つつ あら む'],
 ['て 遊び けむ 時 の', '盛り を 留 みかね 過', '恋 ひ つつ あら む に'],
 ['ぐし つれ 蜷 の', '腸 か黒き 髪 

In [150]:
def train_disc(disc, epochs, trn_dl, val_dl, opt):
    for epoch in range(epochs):
        with tqdm(total=len(trn_dl)) as pbar:
            disc.train()
            for p in disc.parameters(): p.data.clamp_(-0.01, 0.01)
            for i, ds in enumerate(trn_dl):
                x, y = ds
                disc.zero_grad()
                real_loss, attn_r = disc(x)
                fake_loss, attn_f = disc(y)
                disc_loss = (1 - real_loss + fake_loss).mean(0)
                #gen_loss = vae_loss(fake,y,gen.kld_weight)
                disc_loss.backward()
                opt.step()
                pbar.update()
            print(f'Epoch {epoch}:')
            print(f'Loss_Real {real_loss.mean(0).data.item()}; Loss_Fake {fake_loss.mean(0).data.item()}; Loss_Disc {disc_loss.data.item()} ')
        disc.eval()
        for i, ds in enumerate(val_dl):
            x, y = ds
            real_loss, _ = disc(x)
            fake_loss, _ = disc(y)
            disc_loss = (1 - real_loss + fake_loss).mean(0)
        print(f'Val Loss_Real {real_loss.mean(0).data.item()}; Val Loss_Fake {fake_loss.mean(0).data.item()}; Loss_Disc {disc_loss.data.item()} ')

In [151]:
train_disc(disc, 5, trn_disc_dl, val_disc_dl, optimizerD)

100%|█████████▉| 5072/5076 [01:10<00:00, 66.60it/s]Epoch 0:
Loss_Real 1.0; Loss_Fake 4.063834057888016e-05; Loss_Disc 4.063834057888016e-05 
100%|██████████| 5076/5076 [01:10<00:00, 71.86it/s]
Val Loss_Real 0.785717248916626; Val Loss_Fake 5.939255515841069e-06; Loss_Disc 0.21428871154785156 
 10%|▉         | 487/5076 [00:07<01:08, 66.74it/s]


KeyboardInterrupt: 

In [152]:
torch.save(disc.state_dict(), PATH/'models/disc_ja3.h5')

In [33]:
disc.load_state_dict(torch.load(PATH/'models/disc_ja3.h5', map_location=lambda storage, loc: storage))

In [34]:
def ppx(input, target):
    sl, bs = target.size()
    sl_in,bs_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:sl]
    return torch.exp(F.cross_entropy(input.view(-1,nc), target.view(-1)))

In [35]:
def train(gen, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, first=True):
    gen_iterations = 0
    
    for epoch in range(epochs):
        gen.train(); disc.train()
        n = len(trn_dl)
        #train loop
        with tqdm(total=n) as pbar:
            for i, ds in enumerate(trn_dl):
                d_iters = 20 if (first and (epoch < 1)) else 5
                g_iters = 20 if (first and (epoch < 2)) else 3
                x, y = ds
                sl, bs = x.size()
                for j in range(d_iters):
                    for p in disc.parameters(): p.data.clamp_(-0.01, 0.01)
                    gen.eval()
                    disc.train()
                    _,fake = gen(x, sampling=True)
                    disc.zero_grad()
                    fake_loss, f_attn = disc(V(fake))
                    real_loss, r_attn = disc(V(y))
                    disc_loss = (1 - real_loss +fake_loss).mean(0)
                    disc_loss.backward()
                    optimizerD.step()
                
                disc.eval(), gen.train()
                fake_, fake_sample = gen(x,y,sampling=True)
                fake = F.log_softmax(fake_,dim=2)
                reward_t, attn_t = disc(y)
                reward_f, attn_f = disc(fake_sample)
                missing = 1 - y==fake_sample
                gen.zero_grad()
                gen_loss = 0
                for i in range(sl):
                    for j in range(bs):
                        gen_loss += -fake[i,j,fake_sample[i,j]] * reward_f.squeeze(1)[j] #* attn_f[i,j]
                gen_loss = V(gen_loss, requires_grad=True)/bs
                gen_loss.backward()
                optimizerG.step()
                gen_iterations += 1
                pbar.update()
        print(f'Epoch {epoch}:')
        print(f'Loss_D {disc_loss.data.item()}; Loss_G {gen_loss.data.item()} Ppx {ppx(fake_,y)}')
        print(f'D_real {real_loss.mean(0).view(1).data.item()}; Loss_D_fake {fake_loss.mean(0).view(1).data.item()}')
        # val loop
        gen.eval(), disc.eval()
        for i, ds in enumerate(val_dl):
            x, y = ds
            sl, bs = x.size()
            gen_loss=0
            fake, fake_sample = gen(x, sampling=True)
            fake_loss, _ = disc(y)
            real_loss, attn_f = disc(fake_sample)
            disc_loss = (1-real_loss+fake_loss).mean(0)
            for i in range(sl):
                for j in range(bs):
                    gen_loss += -fake[i,j,fake_sample[i,j]] * fake_loss.squeeze(1)[j] #* attn_f[i,j]
        print(f'Loss_D_val {disc_loss.data.item()}; Loss_G_val {gen_loss.mean(0).data.item()} Ppx {ppx(fake,y)}')
        print(f'D_real_val {real_loss.mean(0).view(1).data.item()}; Loss_D_fake_val {fake_loss.mean(0).view(1).data.item()}')

In [90]:
gen.load_state_dict(torch.load(PATH/'models/seq2seq_ja.h5', map_location=lambda storage, loc: storage)) 
disc.load_state_dict(torch.load(PATH/'models/disc_ja.h5', map_location=lambda storage, loc: storage)) 

RuntimeError: Error(s) in loading state_dict for Seq2SeqAttention:
	size mismatch for emb_enc.weight: copying a param of torch.Size([5689, 300]) from checkpoint, where the shape is torch.Size([5168, 300]) in current model.
	size mismatch for emb_dec.weight: copying a param of torch.Size([5689, 300]) from checkpoint, where the shape is torch.Size([5168, 300]) in current model.
	size mismatch for out.weight: copying a param of torch.Size([5689, 300]) from checkpoint, where the shape is torch.Size([5168, 300]) in current model.
	size mismatch for out.bias: copying a param of torch.Size([5689]) from checkpoint, where the shape is torch.Size([5168]) in current model.

In [115]:
torch.save(gen.state_dict(), PATH/'models/netG_J.h5')
torch.save(disc.state_dict(), PATH/'models/netD_J.h5')

In [78]:
gen.load_state_dict(torch.load(PATH/'models/netG_J_F.h5', map_location=lambda storage, loc: storage))
disc.load_state_dict(torch.load(PATH/'models/netD_J.h5', map_location=lambda storage, loc: storage))

In [40]:
def produce_out_sampling(val_dl, model, use_cuda=True):
    model.eval()
    out = []
    loss = 0
    #val_dl.sampler=None
    for i, ds in enumerate(val_dl):
        x, y = ds
        bptt, bs = x.size()
        y = y.view(bptt, bs)
        #x = x.long()
        if use_cuda:
            x = x.cuda()
        probs, sample = model(x, sampling=True)
        p = ppx(probs, y)
        preds = np.array(sample)
        for i in range(x.size(1)):
            seed = ' '.join([itos[o] for o in x[:,i] if o not in [0,1,2,3]])
            actual = ' '.join([itos[o] for o in y[:,i] if o not in [0,1,2,3]])
            fake = ' '.join([itos[o] for o in preds[:,i] if o not in [0,1,2,3]])
            out.append([seed,actual,fake])
    print(p.data.item())
    return out

In [41]:
out_gan = produce_out(val_gan_dl, gen)
out_gan2 = produce_out_sampling(val_gan_dl, gen)
pickle.dump(out_gan, (PATH/'v_gan_ja.pkl').open('wb'))

tensor(772.9237, device='cuda:0', grad_fn=<ExpBackward>)
810.3218994140625


In [164]:
out_gan = pickle.load((PATH/'v_gan_ja.pkl').open('rb'))
out_gan

[['霍公 鳥 聞け ども 飽か', 'ず 網 捕り に 捕り', '高城 むにまされる 廻れ 主人 利 卿'],
 ['鳥 聞け ども 飽か ず', '網 捕り に 捕り て', '捕 みあえ 取り 月日 増し ぐさ'],
 ['聞け ども 飽か ず 網', '捕り に 捕り て', 'おこせ ぶら つとに ぬら 釣船 曰'],
 ['ども 飽か ず 網 捕り', 'に 捕り て な', '御船 橋 衿 杉 花 いら'],
 ['飽か ず 網 捕り に', '捕り て な 離れ', '関山 縦 裾 統 来せ 黒く'],
 ['ず 網 捕り に 捕り', 'て な 離れ ず', '都路 くみ いぶせく 紐解く 巻向 薄れ'],
 ['網 捕り に 捕り て', 'な 離れ ず 鳴く', '壇 知らし ぎかねつひさかたの かくし 散 ゆくゆく'],
 ['捕り に 捕り て', 'な 離れ ず 鳴く が', '大滝 づしその わかめ ぶら 巻向 ゑずて'],
 ['に 捕り て な', '離れ ず 鳴く が ね', '御子 剣 伏 おし 弓矢 武田'],
 ['捕り て な 離れ', 'ず 鳴く が ね やすみ', '山の上 かた 呼び声 荒ぶ あく はや'],
 ['て な 離れ ず', '鳴く が ね やすみ し', '輪 ねり 多里 振れ 司馬 濡れ'],
 ['な 離れ ず 鳴く', 'が ね やすみ し し', '申 人見 移ろ 波立た 軍 さだ'],
 ['な 離れ ず 鳴く が', 'ね やすみ し し 我が', 'よみ 絵 副 よむ ぎぬべしこちごちの もつれ'],
 ['離れ ず 鳴く が ね', 'やすみ し し 我が 大君', 'けども はず むにわたつみの むため 壇 慕'],
 ['ず 鳴く が ね やすみ', 'し し 我が 大君 の', '阿須 立ち上る かめ ふく 在り 入野'],
 ['鳴く が ね やすみ し', 'し 我が 大君 の 敷き', 'ごとき 布勢 はとこ 一節 放れ'],
 ['が ね やすみ し し', '我が 大君 の 敷き ませる', '道行き 枯らさ 着せよ 泊め 増せる 間道'],
 ['ね やすみ し し 我が', '大君 の 敷き ませる 国', '淵 淡雪 むぬばたまの 歌人 設け 墨'],
 ['やす

In [42]:
out_gan2

[['霍公 鳥 聞け ども 飽か', 'ず 網 捕り に 捕り', '忘れ ず 見れ ば 継ぎ'],
 ['て な 離れ ず', '鳴く が ね やすみ し', '偲 はむ 行く 白波 の'],
 ['し 我が 大君 の 敷き', 'ませる 国 の 中 に', 'ませ 見れ ば 霧 刈り'],
 ['は 都 し 思 ほ', 'ゆ 山 高み 谷 辺', 'ゆる かも 見 が 恋'],
 ['に 延 へる 玉葛 絶', 'ゆる 時 なく 見 むよしもがも', 'ゆる まで に 天地 の'],
 ['我が 宿 に 植 ゑ', '生 ほし たる 秋 萩', 'は 今 恋 ら む'],
 ['を 誰 れ か 標', '刺す 我 れ に 知ら', '更け に し ら む'],
 ['え ず お ほなら ばか', 'も かもせ む を 畏', 'は 川 楫 し し'],
 ['み と 振り たき 袖', 'を 忍び て ある かも', 'に 見れ ども は 知ら'],
 ['霰 降り 遠 つ 淡海', 'の 吾 跡 川 楊', 'を 潮 の 玉 ふ'],
 ['刈れ ども また も 生', 'ふと いふ 吾 跡 川', 'ひ ば り ま は'],
 ['楊 世の中 は 恋 繁し', '梅 の 花 に', 'し その 夜 の 音'],
 ['も なら まし もの を', '守 大伴 大夫 世間', '漕ぎ 立ち も 知ら む'],
 ['の すべ なき もの は', '年月 は 流る る ごとし', '我 れ 我 れ は'],
 ['とり 続き 追 ひ 来る', 'もの は 百 種 に', '木 娘 の 来 まし'],
 ['迫 め 寄り 来る 娘', '子 ら が 娘 子', 'を を み 今 も'],
 ['さ 唐 玉 を', '手本 に 巻か し 白妙', 'しき の か も 宿'],
 ['の 袖 振り 交 は', 'し 紅 の 赤 裳裾', 'が 風 の 妹 は'],
 ['引き よ ち 子 ら', 'と 手 携 は り', 'が とも 行く 盛り が'],
 ['て 遊び けむ 時 の', '盛り を 留 みかね 過', '恋 ひ つつ ある か'],
 ['ぐし つれ 蜷 の', '腸 か黒き 髪 に', '国 は 秋 ば 奈良'],
 ['間 か 霜 の 降り', 